In [2]:
import pandas as pd
import altair as alt

prices_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_prices.csv')
prices_df.sample(5)
items_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_item_clean.csv')
items_df
region_df = pd.read_csv("https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_region.csv")
region_df

full_df = pd.merge(prices_df, items_df[['item_id', 'description']], on='item_id') # merge prices and items
full_df

full_df['quote_date'] = pd.to_datetime(full_df['quote_date'], format='%Y%m') # convert quote_date to datetime, format is yearmonth
full_df

items_df[items_df.description.str.contains('turkey', case=False)].sort_values(by=["date_quote_e", "date_quote_s"]) #using this function to find the ID's of the products that im interested in.



,item_id,description,date_quote_s,date_quote_e,n_obs
80,210908,COOKED MEAT-TURKEY-SLICED,199602,200301,21312
76,210904,FROZEN TURKEY- PER KG- SPECIFY,198802,200401,18669
102,211018,COOKED MEAT - SLICED TURKEY,200302,201601,30901
83,210911,FRESH TURKEY STEAKS PER KG,200402,202001,10135
112,211029,COOKED MEAT TURKEY/CHICK SLICE,201602,202409,32247
86,210914,FRESH TURKEY DICED/MIN PER KG,202002,202409,2678


In [4]:


#the different meats im going to compare have the codes:
#Turkey: 210904
#Beeef: 210416
target_item_ids = [210904, 210416]

df = full_df[full_df['item_id'].isin(target_item_ids)] # filter for target item

# Extract year and month instead of just year
df['year_month'] = df['quote_date'].dt.to_period('M')

# Group by year_month and item_id to calculate the median price
grouped_df = df.groupby(['year_month', 'item_id']).agg({'price': 'median'}).reset_index()

#Generating the csv file
pivot_df = grouped_df.pivot(index='year_month', columns='item_id', values='price').reset_index()

pivot_df.rename(columns={210904: 'Turkey', 210416: 'Beef'}, inplace=True)

pivot_df.to_csv('meat_prices_by_month.csv', index=False)

# the final DataFrame
print(pivot_df)



item_id year_month   Beef  Turkey
0          1988-02    NaN    0.66
1          1988-03    NaN    0.65
2          1988-04    NaN    0.63
3          1988-05    NaN    0.61
4          1988-06    NaN    0.62
..             ...    ...     ...
235        2024-05  13.75     NaN
236        2024-06  13.80     NaN
237        2024-07  13.00     NaN
238        2024-08  13.75     NaN
239        2024-09  13.99     NaN

[240 rows x 3 columns]


<ipython-input-4-6a2d2f1c8deb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = df['quote_date'].dt.to_period('M')
